# Out-of-Sample Backtest

This notebook runs a reproducible out-of-sample backtest using the reference optimized parameters. It compares the performance of the optimized parameters against the default parameters.

In [ ]:
import sys
sys.path.append('..')

import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from src.data.data_loader import DataLoader
from src.optimization.config_loader import ConfigLoader
from src.pipeline import TradingPipeline
from src.backtest.performance import PerformanceMetrics
from src.visualization.comparison import plot_parameter_comparison, plot_equity_curves_comparison
from src.visualization.backtest import plot_backtest_results, plot_equity_curve, plot_trade_analysis

RESULTS_DIR = os.path.join('..', 'results', 'outsample')
os.makedirs(RESULTS_DIR, exist_ok=True)

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Define Out-of-Sample Period and Parameter Source

Set the start and end dates for the out-of-sample testing period and choose which optimized parameters to use.

In [ ]:
test_start_date = '2024-06-01'
test_end_date = '2025-01-01'

# Choose whether to use reference parameters (True) or custom optimized parameters (False)
use_reference_parameters = True

print(f"Out-of-sample testing period: {test_start_date} to {test_end_date}")
print(f"Using {'reference' if use_reference_parameters else 'custom'} optimized parameters")

## 2. Load Configurations

Load both the default configuration and the optimized parameters from config files.

In [ ]:
config_loader = ConfigLoader()
default_config = config_loader.get_config()


if use_reference_parameters:
    optimized_config_path = os.path.join('..', 'config', 'reference_optimized_parameters.json')
    if not os.path.exists(optimized_config_path):
        print(f"Reference optimized parameters file not found at {optimized_config_path}")
        print("Using latest optimized parameters instead.")
        optimized_config_path = os.path.join('..', 'config', 'optimized_parameters.json')
else:
    optimized_config_path = os.path.join('..', 'config', 'optimized_parameters.json')

with open(optimized_config_path, 'r') as f:
    optimized_config = json.load(f)

optimized_params = optimized_config['parameters']

print(f"Loading optimized parameters from: {optimized_config_path}")
print("\nOptimized Parameters:")
for param, value in optimized_params.items():
    if param not in ['trailing_trigger', 'trailing_atr', 'trading_start', 'trading_end', 'market_close_time']:
        print(f"- {param}: {value}")

print("\nDefault Parameters:")
default_params = default_config['parameters']
for param, value in default_params.items():
    if param not in ['trailing_trigger', 'trailing_atr', 'trading_start', 'trading_end', 'market_close_time']:
        print(f"- {param}: {value}")

## 3. Run Backtest with Optimized Parameters

Execute the backtest using the optimized parameters on the out-of-sample period.

In [ ]:
print("Running backtest with optimized parameters...")

temp_optimized_config = default_config.copy()
temp_optimized_config['parameters'] = optimized_params

optimized_pipeline = TradingPipeline(temp_optimized_config)

optimized_timeframe = optimized_params.get('default_timeframe', '15min')

optimized_results, optimized_signals_df = optimized_pipeline.run_backtest(
    test_start_date, test_end_date, optimized_timeframe)

In [ ]:
if optimized_results and 'trades' in optimized_results and not optimized_results['trades'].empty:
    optimized_performance = PerformanceMetrics(
        optimized_results['trades'],
        optimized_results['portfolio_history'],
        risk_free_rate=default_config.get('backtest', {}).get('risk_free_rate', 0.03)
    )
    optimized_metrics = optimized_performance.generate_report()
    
    print("Performance with Optimized Parameters:")
    for metric, value in optimized_metrics.items():
        print(f"{metric}: {value:.4f}" if isinstance(value, (int, float)) else f"{metric}: {value}")
else:
    optimized_metrics = None
    print("No trades were executed with optimized parameters")

## 4. Visualize Optimized Results

Generate charts and visualizations of the backtest results with optimized parameters.

In [ ]:
if optimized_results and 'trades' in optimized_results and not optimized_results['trades'].empty:
    fig = plot_backtest_results(
        optimized_signals_df,
        trades_df=optimized_results['trades'],
        save_path=os.path.join(RESULTS_DIR, 'optimized_backtest_chart.png')
    )
    plt.show()
else:
    print("No visualization available - no trades were executed with optimized parameters")

In [ ]:
if optimized_results and 'portfolio_history' in optimized_results and len(optimized_results['portfolio_history']) > 1:
    fig = plot_equity_curve(
        optimized_results['portfolio_history'],
        save_path=os.path.join(RESULTS_DIR, 'optimized_equity_curve.png')
    )
    plt.show()
else:
    print("No equity curve available - insufficient data")

In [ ]:
if optimized_results and 'trades' in optimized_results and not optimized_results['trades'].empty:
    fig_trades, fig_exit, profit_by_exit = plot_trade_analysis(
        optimized_results['trades'],
        save_path=os.path.join(RESULTS_DIR, 'optimized_trade_analysis')
    )
    
    if fig_trades:
        plt.figure(fig_trades.number)
        plt.show()
    
    if fig_exit:
        plt.figure(fig_exit.number)
        plt.show()
    
    if profit_by_exit is not None:
        print("\nProfit by Exit Reason:")
        display(profit_by_exit)
else:
    print("No trade analysis available - no trades were executed with optimized parameters")

## 5. Run Backtest with Default Parameters

Execute the backtest using the default parameters on the out-of-sample period for comparison.

In [ ]:
print("Running backtest with default parameters for comparison...")

default_pipeline = TradingPipeline(default_config)
default_timeframe = default_params.get('default_timeframe', '15min')

default_results, default_signals_df = default_pipeline.run_backtest(
    test_start_date, test_end_date, default_timeframe)

In [ ]:
if default_results and 'trades' in default_results and not default_results['trades'].empty:
    default_performance = PerformanceMetrics(
        default_results['trades'],
        default_results['portfolio_history'],
        risk_free_rate=default_config.get('backtest', {}).get('risk_free_rate', 0.03)
    )
    default_metrics = default_performance.generate_report()
    
    print("Performance with Default Parameters:")
    for metric, value in default_metrics.items():
        print(f"{metric}: {value:.4f}" if isinstance(value, (int, float)) else f"{metric}: {value}")
    
    fig = plot_equity_curve(
        default_results['portfolio_history'],
        save_path=os.path.join(RESULTS_DIR, 'default_equity_curve.png')
    )
    plt.show()
else:
    default_metrics = None
    print("No trades were executed with default parameters")

## 6. Compare Default vs Optimized Parameters

Generate comparison visualizations to show the difference in performance between the default and optimized parameters.

In [ ]:
if default_metrics and optimized_metrics:
    print("Generating comparison visualizations...")
    
    fig_comparison, comparison_df = plot_parameter_comparison(
        default_metrics, 
        optimized_metrics, 
        save_path=os.path.join(RESULTS_DIR, 'metrics_comparison.png')
    )
    plt.show()
    
    print("\nComparison of Default vs Optimized Parameters:")
    display(comparison_df)
    comparison_df.to_csv(os.path.join(RESULTS_DIR, 'parameter_comparison.csv'))
else:
    print("Cannot compare metrics - insufficient data")

if default_metrics and optimized_metrics and 'portfolio_history' in default_results and 'portfolio_history' in optimized_results:
    fig_curves = plot_equity_curves_comparison(
        default_results['portfolio_history'],
        optimized_results['portfolio_history'],
        save_path=os.path.join(RESULTS_DIR, 'default_vs_optimized.png')
    )
    plt.show()
else:
    print("Cannot compare equity curves - insufficient data")

## 7. Save Results

Save the out-of-sample backtest results to a JSON file for future reference.

In [ ]:
if optimized_metrics:
    result_data = {
        "test_period": {
            "start_date": test_start_date,
            "end_date": test_end_date
        },
        "optimized_source": "reference" if use_reference_parameters else "custom",
        "optimized_parameters": optimized_params,
        "optimized_metrics": {
            k: float(v) if isinstance(v, (int, float, np.number)) else v for k, v in optimized_metrics.items()
        },
        "default_metrics": {
            k: float(v) if isinstance(v, (int, float, np.number)) else v for k, v in default_metrics.items()
        } if default_metrics else None,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    with open(os.path.join(RESULTS_DIR, 'outsample_backtest_results.json'), 'w') as f:
        json.dump(result_data, f, indent=4)
    
    print(f"Out-of-sample backtest results saved to {RESULTS_DIR}/outsample_backtest_results.json")
else:
    print("No results saved - insufficient data")